<a href="https://colab.research.google.com/github/venelink/inferes/blob/main/code/BETO_Base.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pandas==1.3.0
import pandas as pd
import random
import nltk

!pip install openpyxl==3.0.0

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

!pip install transformers
from transformers import BertTokenizer, BertModel, BertForSequenceClassification, TFBertForSequenceClassification, XLMRobertaTokenizer, TFXLMRobertaForSequenceClassification
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification

import tensorflow as tf
from tensorflow.keras.optimizers.schedules import PolynomialDecay
from tensorflow.keras.losses import SparseCategoricalCrossentropy


!pip install datasets
from datasets import load_dataset

import numpy as np

from google.colab import drive

from sklearn.metrics import accuracy_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
# Get the data
!git clone https://github.com/venelink/inferes

Cloning into 'inferes'...
remote: Enumerating objects: 26, done.
remote: Counting objects: 100% (26/26), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 26 (delta 3), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (26/26), done.


In [3]:
!ls

inferes  sample_data


In [4]:
inf_train = pd.read_csv('/content/inferes/data/inferes_train.csv')
inf_test = pd.read_csv('/content/inferes/data/inferes_test.csv')

In [6]:
inf_train.head()

,ID,Premise,Hypothesis,Label,Topic,Anno,Anno_Type
0,1968,"Como resultado, los Juegos Olímpicos se han al...","Es poco probable que, antes de que las Olimpia...",cnt,4,2,Generate
1,2017,El COI permite la formación de comités olímpic...,El COI demanda los mismos estrictos requisitos...,cnt,4,2,Generate
2,6580,La Unión de Repúblicas Socialistas Soviéticas ...,"Más de quince repúblicas formaban la URSS, un ...",cnt,6,5,Rewrite
3,5178,"Todos los países del espacio de Schengen, con ...",A finales de los 80 los videojuegos empezaron ...,neutral,5,14,Automated
4,941,El proyecto de Colón fue sometido a una nueva ...,El proyecto del descubridor fue comúnmente apr...,cnt,2,2,Generate


In [7]:
# Model checkpoint
checkpoint = 'dccuchile/bert-base-spanish-wwm-uncased'

# Initialize tokenizer
tokenizer = AutoTokenizer.from_pretrained(checkpoint)



Downloading:   0%|          | 0.00/310 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/650 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/248k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/486k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/134 [00:00<?, ?B/s]

In [9]:
# Split train and dev
inf_train, inf_val = train, test = train_test_split(inf_train, test_size=0.2, random_state=42, stratify=inf_train[['Label']])

# Tokenize the data
inf_train_data = tokenizer(inf_train["Premise"].tolist(),inf_train["Hypothesis"].tolist(),truncation=True, padding=True, return_tensors='np').data
inf_val_data = tokenizer(inf_val["Premise"].tolist(),inf_val["Hypothesis"].tolist(),truncation=True, padding=True, return_tensors='np').data
inf_test_data = tokenizer(inf_test["Premise"].tolist(),inf_test["Hypothesis"].tolist(),truncation=True, padding=True, return_tensors='np').data

# Convert the labels to integers
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(inf_train["Label"])
inf_train["y"] = le.transform(inf_train["Label"])
inf_val["y"] = le.transform(inf_val["Label"])
inf_test["y"] = le.transform(inf_test["Label"])

In [10]:
model = TFAutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=3)

batch_size = 8
num_epochs = 5
# The number of training steps is the number of samples in the dataset, divided by the batch size then multiplied
# by the total number of epochs
num_train_steps = (len(inf_train["y"]) // batch_size) * num_epochs
lr_scheduler = PolynomialDecay(
    initial_learning_rate=5e-5,
    end_learning_rate=0.,
    decay_steps=num_train_steps
    )
from tensorflow.keras.optimizers import Adam
opt = Adam(learning_rate=lr_scheduler)
loss = SparseCategoricalCrossentropy(from_logits=True)

model.compile(optimizer=opt, loss=loss, metrics=['accuracy'])
model.fit(
    inf_train_data,
    np.array(inf_train["y"]), 
    validation_data=(
        inf_val_data,
        np.array(inf_val["y"]),
    ),
    batch_size=batch_size,
    epochs = num_epochs
)

Downloading:   0%|          | 0.00/537M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-uncased and are newly initialized: ['classifier', 'bert/pooler/dense/bias:0', 'bert/pooler/dense/kernel:0']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/5
516/516 [==============================] - 235s 419ms/step - loss: 0.7824 - accuracy: 0.6125 - val_loss: 0.7143 - val_accuracy: 0.6537
Epoch 2/5
516/516 [==============================] - 218s 423ms/step - loss: 0.5767 - accuracy: 0.7420 - val_loss: 0.6975 - val_accuracy: 0.6809
Epoch 3/5
516/516 [==============================] - 219s 425ms/step - loss: 0.3867 - accuracy: 0.8363 - val_loss: 0.7771 - val_accuracy: 0.6867
Epoch 4/5
516/516 [==============================] - 219s 425ms/step - loss: 0.2228 - accuracy: 0.9117 - val_loss: 0.8970 - val_accuracy: 0.6974
Epoch 5/5
516/516 [==============================] - 219s 425ms/step - loss: 0.1160 - accuracy: 0.9539 - val_loss: 1.1447 - val_accuracy: 0.6974


In [12]:
# Predict on test
preds = model.predict(inf_test_data)['logits']
class_preds = np.argmax(preds, axis=1)
print(accuracy_score(inf_test["y"],class_preds))

0.6811414392059554
